In [ ]:
%load_ext autoreload
%autoreload 2
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import interp1d
from scipy.signal import savgol_filter

from pasna_analysis import Experiment, Embryo, Trace

experiment_name = '20240515'
exp_path = Path.cwd().parent.joinpath('data',experiment_name)
exp = Experiment(exp_path)

In [ ]:
a, l = exp.get_embryo_files_by_id(3)
embryo = Embryo(a, l)

fig, ax = plt.subplots(figsize=(10,6))
time = embryo.activity[:, 0]/60
act = embryo.activity[:, 1]
stct = embryo.activity[:, 2]

trace = Trace(time, act, stct)
dff = trace.compute_dff()
trace.detect_peaks()
time = time[:trace.trim_idx]

savgol = trace.savgol
savgol1 = trace.savgol1
savgol2 = trace.savgol2

ax.plot(time, dff, label='dF/F')
ax.plot(time, savgol, label='savgol')
ax.plot(time, savgol1, label='savgol1')
ax.plot(time, savgol2, label='savgol2')
ax.set_xlabel('Time [min]')
ax.legend()

for x in trace.peak_idxes:
    ax.scatter(time[x], savgol[x], color='k')

In [ ]:
'''Visualizing peak values, developmental time at peak value times, and 
the relationship between dff and dev time.'''
lengths = exp.lengths()
activities = exp.activity()

embryos = [Embryo(act, length) for act, length in zip(activities, lengths)]

fig, (ax, ax2, ax3, ax4) = plt.subplots(4, figsize=(10,16))
for emb in embryos:
    print(emb.name)
    # estimate developmental time for all t points with interp1d
    dvt = emb.developmental_time()
    dvt_timepoints = emb.vnc_length[:, 0]
    interpolator = interp1d(dvt_timepoints, dvt, kind='linear', fill_value='extrapolate')

    time, act, stct = emb.activity[:, 0], emb.activity[:, 1], emb.activity[:, 2], 
    trace = Trace(time, act, stct)
    first_peak = trace.get_first_peak_time()
    if first_peak < 30*60:
        print('First peak detected before 30 mins of imaging.')
    else:
        time = time[:trace.trim_idx]
        dev_time = emb.get_DT_from_time(time)
        if dev_time[-1] < 0:
            continue
        first_peak_dev = emb.get_DT_from_time(first_peak)

        interval = time > (first_peak - 300)
        ax.plot(time, trace.dff)
        ax2.plot(time[interval] - first_peak, trace.dff[interval])

        interval = (dev_time > first_peak_dev - 0.01)
        ax3.plot(dev_time, trace.dff)
        ax4.plot(dev_time[interval] - first_peak_dev, trace.dff[interval])

ax.set_title('Dff x time')
ax2.set_title('Dff x time, aligned by first peak')
ax3.set_title('Dff x developmental time')
ax4.set_title('Dff x developmental time, aligned by first peak')
fig.tight_layout()

In [ ]:
'''What was the developmental time in the beginning of the movie and 
before hatching?'''
lengths = exp.lengths()
activities = exp.activity()

embryos = [Embryo(act, length) for act, length in zip(activities, lengths)]

fig, ax = plt.subplots()

peak_times = []
dev_firsts = []
firsts = []
dev_ends = []
ends = []

for emb in embryos:
    time, act, stct = emb.activity[:, 0], emb.activity[:, 1], emb.activity[:, 2], 
    trace = Trace(time, act, stct)

    trace.detect_peaks()
    time = time[:trace.trim_idx]
    time_first_peak = trace.peak_times[0]

    if first_peak < 30*60:
        print(f'First peak found before 30 mins. Skipping {emb.name}..')
        continue

    dev_time_first_peak = emb.get_DT_from_time(time_first_peak)
    dev_time_end = emb.get_DT_from_time(time[-1])

    dev_firsts.append(dev_time_first_peak*5000)
    firsts.append(time_first_peak)
    dev_ends.append(dev_time_end*5000)
    ends.append(time[-1])

ax.scatter(np.zeros_like(dev_firsts), dev_firsts, label='dev_first')
ax.scatter(np.zeros_like(firsts), firsts, label='first')
ax.scatter(np.zeros_like(dev_ends)+10, dev_ends, label='dev_end')
ax.scatter(np.zeros_like(ends)+10, ends, label='end')

ax.set_title('Developmental time')
ax.legend()

In [ ]:
'''Different ways to smooth dev time.'''
from scipy.ndimage import gaussian_filter1d

def moving_average(data, window_size):
    return np.convolve(data, np.ones(window_size)/window_size, mode='valid')

lengths = exp.lengths()
activities = exp.activity()

embryos = [Embryo(act, length) for act, length in zip(activities, lengths)]

fig, ax = plt.subplots()

emb = embryos[19]
# estimate developmental time for all t points with interp1d
dvt = emb.developmental_time()
dvt_timepoints = emb.vnc_length[:, 0]

savgol = savgol_filter(dvt, window_length=5, polyorder=1)
gaussian = gaussian_filter1d(dvt, sigma=5)
mov_av = moving_average(dvt, 5)

ax.plot(dvt_timepoints/60, dvt, label='raw')
ax.plot(dvt_timepoints/60, savgol, label='savgol')
ax.plot(dvt_timepoints/60, gaussian, label='gaussian')
ax.set_title('Approximations for VNC length')
ax.legend()
fig.tight_layout()

In [ ]:
'''Align by developmental time'''

lengths = exp.lengths()
activities = exp.activity()

embryos = [Embryo(act, length) for act, length in zip(activities, lengths)]
embryos = embryos[10:20]

fig, (ax, ax2) = plt.subplots(2, figsize=(14, 10))

end_times = []
peak_times = []
dev_zeros = []
dev_firsts = []
dev_end = []
traces = []
trace_times = []

for emb in embryos:
    # estimate developmental time for all t points with interp1d
    dvt = emb.developmental_time()
    dvt_timepoints = emb.vnc_length[:, 0]/60
    interpolator = interp1d(dvt_timepoints, dvt, kind='linear', fill_value='extrapolate')

    time, act, stct = emb.activity[:, 0], emb.activity[:, 1], emb.activity[:, 2], 
    time = time / 60
    trace = Trace(time, act, stct)
    trace.detect_peaks()
    time = time[:trace.trim_idx]
    first_peak = trace.peak_times[0]
    end = interpolator(time[-1])
    # if end > 3 or end < 1.5:
    #     continue
    dev_time_first_peak = interpolator(first_peak)
    dev_firsts.append(dev_time_first_peak)
    dev_zeros.append(interpolator(0))
    dev_end.append(interpolator(time[-1]))
    peak_times.append(first_peak)
    end_times.append(time[-1])
    traces.append(trace.dff)
    trace_times.append(time)
    
last_hatching = max(end_times)
diffs = [last_hatching - t for t in end_times]

adjusted_peak_times = [t+d for t, d in zip(peak_times, diffs)]
adjusted_zeros = diffs
end_times = np.ones_like(dev_end)*last_hatching
        
ax.scatter(adjusted_zeros, dev_zeros, label='t=0')
ax.scatter(adjusted_peak_times, dev_firsts, label='t=first_peak')
ax.scatter(end_times, dev_end, label='t=before_hacthing')
for i, _ in enumerate(dev_zeros):
    ax.plot((adjusted_zeros[i], adjusted_peak_times[i], last_hatching),(dev_zeros[i], dev_firsts[i], dev_end[i]),  color='k', linewidth=0.2)
ax.set_title('Developmental time')
ax.legend()

for i, emb in enumerate(embryos):
    ax2.plot(trace_times[i]+diffs[i], traces[i], linewidth=0.8)

fig.tight_layout()